currently this is our implementation as of now

this is a small trail to know hoe deep neural networks works for predicting dosha


we contact ayurveda dept. and currently on the way to collect data to predict the dosha from them for training  purposes. also they sharead a process of finding dosha thouthg input score

Now the mail goal diverts to the point of making #D models as required form teh input of user and intergrating it with the backend for the final achievement

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Load your data
data = pd.read_csv('data1.csv')

# Separate features and target
X = data.drop(columns=['C'])
y = data['C']

# One-hot encode categorical features
encoder = OneHotEncoder(sparse=False)
X_encoded = encoder.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.15, random_state=30)


c:\Users\LOKESH\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
y_test

42      Vata
86     Kapha
83      Vata
101    Kapha
38     Kapha
68      Vata
103    Kapha
59     Pitta
99      Vata
111     Vata
143     Vata
8      Kapha
35     Kapha
132     Vata
1      Kapha
5      Pitta
20     Kapha
67      Vata
0       Vata
32     Kapha
24      Vata
74      Vata
66      Vata
Name: C, dtype: object

In [5]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch

# Use LabelEncoder to encode the categorical target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Now, y_train_encoded and y_test_encoded contain numerical labels

# Define CustomDataset class with encoded target variable
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # Use torch.long for integer labels
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create instances of CustomDataset
train_dataset = CustomDataset(X_train, y_train_encoded)
test_dataset = CustomDataset(X_test, y_test_encoded)

# DataLoader remains the same
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Adjust input size as per your data
        self.fc2 = nn.Linear(64, output_size)  # Adjust output size as per your data
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyModel(input_size=X_train.shape[1], output_size=len(set(y_train)))


In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    if epoch%10 == 0 : print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')


Epoch [1/100], Loss: 0.0004
Epoch [11/100], Loss: 0.0003
Epoch [21/100], Loss: 0.0003
Epoch [31/100], Loss: 0.0002
Epoch [41/100], Loss: 0.0002
Epoch [51/100], Loss: 0.0002
Epoch [61/100], Loss: 0.0001
Epoch [71/100], Loss: 0.0001
Epoch [81/100], Loss: 0.0001
Epoch [91/100], Loss: 0.0001


In [10]:
model.eval()  # Set model to evaluation mode

# Lists to store predicted and actual labels
predicted_labels = []
actual_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        # Convert predicted labels back to original categorical values
        predicted_labels.extend(label_encoder.inverse_transform(predicted.numpy()))
        actual_labels.extend(label_encoder.inverse_transform(labels.numpy()))

# Print predicted and actual labels
for predicted_label, actual_label in zip(predicted_labels, actual_labels):
    print(f"Predicted: {predicted_label}, Actual: {actual_label}")


#accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(actual_labels, predicted_labels)
print("Accuracy Score: ", accuracy)


Predicted: Vata, Actual: Vata
Predicted: Kapha, Actual: Kapha
Predicted: Vata, Actual: Vata
Predicted: Kapha, Actual: Kapha
Predicted: Kapha, Actual: Kapha
Predicted: Vata, Actual: Vata
Predicted: Kapha, Actual: Kapha
Predicted: Vata, Actual: Pitta
Predicted: Vata, Actual: Vata
Predicted: Vata, Actual: Vata
Predicted: Pitta, Actual: Vata
Predicted: Kapha, Actual: Kapha
Predicted: Kapha, Actual: Kapha
Predicted: Vata, Actual: Vata
Predicted: Kapha, Actual: Kapha
Predicted: Pitta, Actual: Pitta
Predicted: Kapha, Actual: Kapha
Predicted: Vata, Actual: Vata
Predicted: Vata, Actual: Vata
Predicted: Kapha, Actual: Kapha
Predicted: Vata, Actual: Vata
Predicted: Vata, Actual: Vata
Predicted: Vata, Actual: Vata
Accuracy Score:  0.9130434782608695


In [11]:
import torch.nn.functional as F

model.eval()  # Set model to evaluation mode

# Lists to store predicted labels, actual labels, and probabilities
predicted_labels = []
actual_labels = []
probabilities = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        probabilities_batch = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities
        
        _, predicted = torch.max(outputs, 1)
        
        # Convert predicted labels back to original categorical values
        predicted_labels.extend(label_encoder.inverse_transform(predicted.numpy()))
        actual_labels.extend(label_encoder.inverse_transform(labels.numpy()))
        probabilities.extend(probabilities_batch.numpy())

# Print predicted and actual labels along with output probabilities
for predicted_label, actual_label, prob in zip(predicted_labels, actual_labels, probabilities):
    print(f"Predicted: {predicted_label}, Actual: {actual_label}")
    print("Output Probabilities:")
    for category, probability in zip(label_encoder.classes_, prob):
        print(f"{category}: {probability:.4f}")
    print()


Predicted: Vata, Actual: Vata
Output Probabilities:
Kapha: 0.0001
Pitta: 0.0003
Vata: 0.9997

Predicted: Kapha, Actual: Kapha
Output Probabilities:
Kapha: 0.9785
Pitta: 0.0177
Vata: 0.0038

Predicted: Vata, Actual: Vata
Output Probabilities:
Kapha: 0.0001
Pitta: 0.0002
Vata: 0.9998

Predicted: Kapha, Actual: Kapha
Output Probabilities:
Kapha: 1.0000
Pitta: 0.0000
Vata: 0.0000

Predicted: Kapha, Actual: Kapha
Output Probabilities:
Kapha: 1.0000
Pitta: 0.0000
Vata: 0.0000

Predicted: Vata, Actual: Vata
Output Probabilities:
Kapha: 0.0001
Pitta: 0.4746
Vata: 0.5254

Predicted: Kapha, Actual: Kapha
Output Probabilities:
Kapha: 0.9991
Pitta: 0.0008
Vata: 0.0000

Predicted: Vata, Actual: Pitta
Output Probabilities:
Kapha: 0.0000
Pitta: 0.0000
Vata: 1.0000

Predicted: Vata, Actual: Vata
Output Probabilities:
Kapha: 0.0000
Pitta: 0.0000
Vata: 1.0000

Predicted: Vata, Actual: Vata
Output Probabilities:
Kapha: 0.0000
Pitta: 0.0000
Vata: 1.0000

Predicted: Pitta, Actual: Vata
Output Probabilities